In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [33]:
df = pd.read_csv('mta_art_data.csv')

In [34]:
df.head()

,Agency,Station Name,Line,Artist,Art Title,Art Date,Art Material,Art Description,Art Image Link
0,NYCT,125 St,"4,5,6",Houston Conwill,The Open Secret,1986,Bronze - polychromed,"The Open Secret, in the 125th Street and Lexin...",https://new.mta.info/agency/arts-design/collec...
1,NYCT,Astor Pl,6,Milton Glaser,Untitled,1986,Porcelain enamel murals,"Milton Glaser, best known for his work in grap...",https://new.mta.info/agency/arts-design/collec...
2,NYCT,Clark St,"2,3",Ray Ring,Clark Street Passage,1987,Terrazzo floor tile,The first model that Brooklyn-born artist Ray ...,https://new.mta.info/agency/arts-design/collec...
3,NYCT,Kings Hwy,"B,Q",Rhoda Andors,Kings Highway Hieroglyphs,1987,Porcelain Enamel Murals on Steel,The artist discusses her work: ÒIf public art...,https://new.mta.info/agency/arts-design/collec...
4,NYCT,Newkirk Av,"B,Q",David Wilson,Transit Skylight,1988,Zinc-glazed Apolycarbonate skylight,"The artist recalls, ÒAbout the same time that ...",https://new.mta.info/agency/arts-design/collec...


In [35]:
def get_image_link(mta_url):
  try:
    response = requests.get(mta_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    image_tag = soup.find('img')

    if image_tag and 'src' in image_tag.attrs:
      return image_tag['src']
    else:
      return None
    
  except requests.exceptions.RequestException as e:
    print(f"Error fetching {mta_url}: {e}")
    return None

In [36]:
df['direct_image_link'] = df['Art Image Link'].apply(get_image_link)

Error fetching nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error fetching nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error fetching nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error fetching nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error fetching nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?


In [37]:
df[df['direct_image_link'].isnull()]

,Agency,Station Name,Line,Artist,Art Title,Art Date,Art Material,Art Description,Art Image Link,direct_image_link
27,NYCT,207 St,1,Wopo Holup,Elevated Nature I-IV,1991,Cast concrete,From the artist: ÒNature's forms have been th...,NaN,None
28,NYCT,125 St,1,Wopo Holup,River to River,1991,Cast concrete,NaN,NaN,None
61,NYCT,Roosevelt Island,F,Robert Hickman,Convex Disk at Roosevelt Island,1996,Stratified glass squares mounted on a steel ar...,NaN,NaN,None
145,NYCT,Coney Island-Stillwell Av,"D, F, N, Q",Atlantic Terra Cotta Company,BMT Facade on Building Entrance,2005,Terracotta,With the renovation of the Stillwell Avenue Co...,NaN,None
359,NYCT,Lorimer St,L,Chlo‘ Bass,Personal Choice #5,2023,Glass mosaic;,NaN,NaN,None


In [38]:
replacements = {
  'River to River': 'https://images.cf.nycsubway.org/images/i11000/img_11381.jpg',
  'Elevated Nature I-IV': 'https://images.cf.nycsubway.org/images/i79000/img_79192.jpg',
  'Convex Disk at Roosevelt Island': 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Roosevelt_Island_IND_td_%282019-11-03%29_13_-_Station_House%2C_Convex_Disk.jpg/800px-Roosevelt_Island_IND_td_%282019-11-03%29_13_-_Station_House%2C_Convex_Disk.jpg?20191104054214',
  'BMT Facade on Building Entrance': 'https://ephemeralnewyork.wordpress.com/wp-content/uploads/2024/06/stillwellbmtcloseup.jpg?w=1024',
  'Personal Choice #5': 'https://files.mta.info/s3fs-public/styles/large/public/2023-08/Bass_NYCT%20Lorimer%20St_02_%20Nicholas%20Knight.jpeg?itok=O4XvG8Jh'
}

In [39]:
df['direct_image_link'] = df['Art Title'].map(replacements).fillna(df['direct_image_link'])

In [40]:
df[df['direct_image_link'].isnull()]

,Agency,Station Name,Line,Artist,Art Title,Art Date,Art Material,Art Description,Art Image Link,direct_image_link


In [41]:
df.to_csv('clean_art_data.csv', index=False)